In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import sys
sys.path.append("../src")
from utils import read_xray
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

## Train

In [7]:
input_dir = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/train/"
output_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_1024/"

In [8]:
filenames = [i for i in os.listdir(input_dir) if i.endswith('.dicom')]

In [ ]:
os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)

for i in tqdm(filenames):
    original = read_xray(os.path.join(input_dir, i))
    im = resize(original, size = 1024)
    im = im.convert('RGB')
    im.save(os.path.join(output_dir, 'train/')+i[:-6]+'.png', 'PNG', quality= 95)

A Jupyter Widget

## Test

In [4]:
input_dir = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/test/"
output_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_1024/"
filenames = [i for i in os.listdir(input_dir) if i.endswith('.dicom')]

In [7]:
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

for i in tqdm(filenames):
    original = read_xray(os.path.join(input_dir, i))
    im = resize(original, size = 1024)
    im = im.convert('RGB')
    im.save(os.path.join(output_dir, 'test/')+i[:-6]+'.png', 'PNG', quality= 95)

A Jupyter Widget

256X256

### Train

In [6]:
input_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_1024/train"
output_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/"
filenames = [i for i in os.listdir(input_dir) if i.endswith('.png')]

In [13]:
os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)


for i in tqdm(filenames):
    im = Image.open(os.path.join(input_dir, i))
    im = im.resize((256, 256), Image.LANCZOS)
    im.save(os.path.join(output_dir, 'train/')+i, 'PNG', quality= 95)

A Jupyter Widget

### Test

In [14]:
input_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_1024/test"
output_dir = "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/"
filenames = [i for i in os.listdir(input_dir) if i.endswith('.png')]

In [15]:
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)


for i in tqdm(filenames):
    im = Image.open(os.path.join(input_dir, i))
    im = im.resize((256, 256), Image.LANCZOS)
    im.save(os.path.join(output_dir, 'test/')+i, 'PNG', quality= 95)

A Jupyter Widget